In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sqlalchemy as sa

In [2]:
# Database Connection
engine = sa.create_engine('mssql+pyodbc://@PresHacks/AgriDiseaseDB?driver=SQL+Server&trusted_connection=yes')
print("Connected to AgriDiseaseDB")

Connected to AgriDiseaseDB


In [4]:
# Populate Farms
farms_data = []
for i in range(50):
    farms_data.append({
        "farm_name": f"Farm_{i+1}",
        "latitude": np.random.uniform(40.0, 42.0),
        "longitude": np.random.uniform(-100.0, -98.0),
        "soil_type": np.random.choice(["Loam", "Clay", "Sandy"]),
        "irrigation_method": np.random.choice(["Drip", "Flood", "None"]),
        "farm_size_ha": np.random.uniform(5, 50)
    })

farms_df = pd.DataFrame(farms_data)
farms_df.to_sql("Farms", engine, if_exists="append", index=False)
print("Farms populated:", len(farms_df))

Farms populated: 50


In [5]:
# Populate Crops
farm_ids = pd.read_sql("SELECT farm_id FROM Farms", engine)["farm_id"].tolist()
crops_data = []
for farm_id in farm_ids:
    for _ in range(np.random.randint(1, 4)):
        crops_data.append({
            "farm_id": farm_id,
            "crop_type": np.random.choice(["Wheat", "Corn", "Soybean"]),
            "planting_date": datetime(2024, np.random.randint(1, 5), np.random.randint(1, 28))
        })

crops_df = pd.DataFrame(crops_data)
crops_df.to_sql("Crops", engine, if_exists="append", index=False)
print("Crops populated:", len(crops_df))

Crops populated: 106


In [6]:
# Populate DiseaseReports
crop_data = pd.read_sql("SELECT crop_id, farm_id FROM Crops", engine).to_dict("records")
disease_data = []
for crop in crop_data:
    for _ in range(np.random.randint(0, 3)):
        disease_data.append({
            "farm_id": crop["farm_id"],
            "crop_id": crop["crop_id"],
            "disease_name": np.random.choice(["Blight", "Rust", "Mildew"]),
            "report_date": datetime(2024, np.random.randint(1, 13), np.random.randint(1, 28)),
            "severity": np.random.randint(1, 11),
            "description": np.random.choice(["Yellow spots", "Wilting leaves", "Gray patches"])
        })

disease_df = pd.DataFrame(disease_data)
disease_df.to_sql("DiseaseReports", engine, if_exists="append", index=False)
print("DiseaseReports populated:", len(disease_df))

DiseaseReports populated: 107


In [7]:
# Populate WeatherData
dates = [datetime(2024, 1, 1) + timedelta(days=i) for i in range(365)]
weather_data = []
for farm_id in farm_ids:
    for date in dates:
        month = date.month
        weather_data.append({
            "farm_id": farm_id,
            "record_date": date,
            "temperature_c": np.random.normal(20 + 5 * np.sin(month * np.pi / 6), 3),
            "humidity_percent": np.random.uniform(50, 90),
            "rainfall_mm": np.random.exponential(10 if month in [6, 7, 8] else 5),
            "wind_speed_kph": np.random.uniform(5, 20)
        })

weather_df = pd.DataFrame(weather_data)
weather_df.to_sql("WeatherData", engine, if_exists="append", index=False)
print("WeatherData populated:", len(weather_df))

WeatherData populated: 18250


In [8]:
# Verify Data
for table in ["Farms", "Crops", "DiseaseReports", "WeatherData"]:
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine)["count"].iloc[0]
    print(f"{table} has {count} rows")

Farms has 50 rows
Crops has 106 rows
DiseaseReports has 107 rows
WeatherData has 18250 rows
